### imports

In [1]:
import polars as pl

pl.Config.set_tbl_rows(20)
pl.Config.set_fmt_str_lengths(100)

from polars import col as c
import os
import matplotlib.pyplot as plt
import numpy as np
import holidays
from datetime import datetime, timedelta
from numpy.typing import ArrayLike, NDArray

### constants

In [2]:
DATA_DIR = "dataset"
US_HOLIDAYS = holidays.US()  # this is a dict-like object

### functions

In [3]:
def mape_f(y_true: ArrayLike, y_pred: ArrayLike) -> np.floating:
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    metric = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return round(metric, 2)

### data

In [4]:
path = os.path.join("../", DATA_DIR, "train.csv")
train = pl.read_csv(path).with_columns(
    c("pickup_date").str.to_datetime("%Y-%m-%d %H:%M:%S")
)
path = os.path.join("../", DATA_DIR, "validation.csv")
validation = pl.read_csv(path).with_columns(
    c("pickup_date").str.to_datetime("%Y-%m-%d %H:%M:%S")
)
path = os.path.join("../", DATA_DIR, "test.csv")
test = pl.read_csv(path).with_columns(
    c("pickup_date").str.to_datetime("%Y-%m-%d %H:%M:%S")
)
train.sample()

rate,valid_miles,transport_type,weight,pickup_date,origin_kma,destination_kma
f64,f64,str,f64,datetime[μs],str,str
3.1834,409.536,"""MKPFX""",19430.0,2021-11-17 16:12:00,"""OKPES""","""NTODX"""


### fill null

In [5]:
train.null_count()

rate,valid_miles,transport_type,weight,pickup_date,origin_kma,destination_kma
u32,u32,u32,u32,u32,u32,u32
0,0,0,80,0,0,0


In [6]:
validation.null_count()

rate,valid_miles,transport_type,weight,pickup_date,origin_kma,destination_kma
u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0


In [7]:
test.null_count()

valid_miles,transport_type,weight,pickup_date,origin_kma,destination_kma
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


In [8]:
train = train.with_columns(c("weight").fill_null(strategy="mean"))

### date features

In [9]:
def is_holiday_next_days(x):
    if x + timedelta(days=1) in US_HOLIDAYS:
        return 1
    elif x + timedelta(days=2) in US_HOLIDAYS:
        return 1
    elif x + timedelta(days=3) in US_HOLIDAYS:
        return 1
        # elif x + timedelta(days=4) in US_HOLIDAYS:
        #     return 1
        # elif x + timedelta(days=5) in US_HOLIDAYS:
        #     return 1
        # elif x + timedelta(days=6) in US_HOLIDAYS:
        return 1
    else:
        return 0

In [10]:
def add_date_features(df: pl.DataFrame) -> pl.DataFrame:
    df = df.with_columns(
        # make trigonometric
        c("pickup_date").dt.month().alias("month"),
        c("pickup_date").dt.weekday().alias("weekday"),
        c("pickup_date").dt.week().alias("week"),
        # c("pickup_date").dt.hour().alias("hour"),
        #
        c("pickup_date").dt.year().alias("year"),
        c("pickup_date").dt.date().alias("date"),
        #
        # holidays
        c("pickup_date")
        .dt.date()
        .map_elements(
            function=is_holiday_next_days,
            return_dtype=pl.Int64,
        )
        .alias("is_holiday_next_week"),
        c("pickup_date")
        .dt.date()
        .map_elements(
            function=lambda x: 1 if x in US_HOLIDAYS else 0,
            return_dtype=pl.Int64,
        )
        .alias("is_holiday"),
    )
    return df

In [11]:
train = add_date_features(train)
validation = add_date_features(validation)
test = add_date_features(test)
train.head()

rate,valid_miles,transport_type,weight,pickup_date,origin_kma,destination_kma,month,weekday,week,year,date,is_holiday_next_week,is_holiday
f64,f64,str,f64,datetime[μs],str,str,i8,i8,i8,i32,date,i64,i64
4.7203,521.8451,"""MKPFX""",9231.75,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0
4.9005,532.6675,"""MKPFX""",11754.95,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0
4.7018,523.9188,"""MKPFX""",9603.2,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0
4.6388,562.8296,"""MKPFX""",8789.05,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0
5.0206,519.8782,"""MKPFX""",9597.5,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0


### weight/mile features

In [12]:
def add_weight_mile_features(df: pl.DataFrame) -> pl.DataFrame:
    df = df.with_columns(
        (c("weight") * c("valid_miles")).alias("weight_mile"),
        (c("weight") / c("valid_miles")).alias("weight_per_mile"),
        c("valid_miles").log(),
    )
    return df

In [13]:
train = add_weight_mile_features(train)
validation = add_weight_mile_features(validation)
test = add_weight_mile_features(test)
train.head()

rate,valid_miles,transport_type,weight,pickup_date,origin_kma,destination_kma,month,weekday,week,year,date,is_holiday_next_week,is_holiday,weight_mile,weight_per_mile
f64,f64,str,f64,datetime[μs],str,str,i8,i8,i8,i32,date,i64,i64,f64,f64
4.7203,6.257371,"""MKPFX""",9231.75,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,4.8175e6,17.690594
4.9005,6.277897,"""MKPFX""",11754.95,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,6.2615e6,22.068082
4.7018,6.261337,"""MKPFX""",9603.2,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,5.0313e6,18.329558
4.6388,6.332977,"""MKPFX""",8789.05,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,4.9467e6,15.615828
5.0206,6.253595,"""MKPFX""",9597.5,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,4.9895e6,18.461055


### kma_pair_feature

In [14]:
def add_kma_pair_feature(df: pl.DataFrame) -> pl.DataFrame:
    df = df.with_columns(
        pl.concat_list("origin_kma", "destination_kma")
        # .list.sort()
        .list.join("_").alias("kma_pair"),
    )
    # .drop("origin_kma", "destination_kma")
    return df

In [15]:
train = add_kma_pair_feature(train)
validation = add_kma_pair_feature(validation)
test = add_kma_pair_feature(test)
train.head()

rate,valid_miles,transport_type,weight,pickup_date,origin_kma,destination_kma,month,weekday,week,year,date,is_holiday_next_week,is_holiday,weight_mile,weight_per_mile,kma_pair
f64,f64,str,f64,datetime[μs],str,str,i8,i8,i8,i32,date,i64,i64,f64,f64,str
4.7203,6.257371,"""MKPFX""",9231.75,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,4.8175e6,17.690594,"""OMUOI_LFUHN"""
4.9005,6.277897,"""MKPFX""",11754.95,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,6.2615e6,22.068082,"""OMUOI_LFUHN"""
4.7018,6.261337,"""MKPFX""",9603.2,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,5.0313e6,18.329558,"""OMUOI_LFUHN"""
4.6388,6.332977,"""MKPFX""",8789.05,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,4.9467e6,15.615828,"""OMUOI_LFUHN"""
5.0206,6.253595,"""MKPFX""",9597.5,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,4.9895e6,18.461055,"""OMUOI_LFUHN"""


### split train into train_df and eval_df

In [16]:
train_df = train.filter(
    c("pickup_date").dt.date() < train["pickup_date"].max().date() - timedelta(days=22)
)
eval_df = train.filter(
    c("pickup_date").dt.date() >= train["pickup_date"].max().date() - timedelta(days=22)
)

### kma features

In [17]:
kma_transport_median = (
    train_df.group_by("kma_pair", "transport_type")
    .agg(
        c("rate").count().alias("kma_transport_count"),
        c("rate").median().alias("kma_transport_median"),
    )
    .sort("kma_transport_count")
    # .filter(c("kma_transport_count") >= 5)
    .drop("kma_transport_count")
)
kma_transport_median.head()

kma_pair,transport_type,kma_transport_median
str,str,f64
"""RPJIS_RJGHA""","""GJROY""",7.1223
"""RCDSS_DLGVW""","""MKPFX""",4.1065
"""NPCXM_TVZUE""","""KFEGT""",8.9591
"""TXLFD_NTQBJ""","""MKPFX""",3.0478
"""RPJIS_QGHCU""","""GJROY""",6.5445


In [18]:
def add_kma_transport_median(df: pl.DataFrame) -> pl.DataFrame:
    df = df.join(
        kma_transport_median,
        on=["kma_pair", "transport_type"],
        how="left",
    ).with_columns(
        c("kma_transport_median").fill_null(0),
    )
    return df

In [19]:
train_df = add_kma_transport_median(train_df)
eval_df = add_kma_transport_median(eval_df)
train = add_kma_transport_median(train)
validation = add_kma_transport_median(validation)
test = add_kma_transport_median(test)
train_df.head()

rate,valid_miles,transport_type,weight,pickup_date,origin_kma,destination_kma,month,weekday,week,year,date,is_holiday_next_week,is_holiday,weight_mile,weight_per_mile,kma_pair,kma_transport_median
f64,f64,str,f64,datetime[μs],str,str,i8,i8,i8,i32,date,i64,i64,f64,f64,str,f64
4.7203,6.257371,"""MKPFX""",9231.75,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,4.8175e6,17.690594,"""OMUOI_LFUHN""",4.6315
4.9005,6.277897,"""MKPFX""",11754.95,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,6.2615e6,22.068082,"""OMUOI_LFUHN""",4.6315
4.7018,6.261337,"""MKPFX""",9603.2,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,5.0313e6,18.329558,"""OMUOI_LFUHN""",4.6315
4.6388,6.332977,"""MKPFX""",8789.05,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,4.9467e6,15.615828,"""OMUOI_LFUHN""",4.6315
5.0206,6.253595,"""MKPFX""",9597.5,2019-11-10 10:42:00,"""OMUOI""","""LFUHN""",11,7,45,2019,2019-11-10,1,0,4.9895e6,18.461055,"""OMUOI_LFUHN""",4.6315


In [20]:
train_df.select("rate", "kma_transport_median").corr()

rate,kma_transport_median
f64,f64
1.0,0.838627
0.838627,1.0


In [21]:
mape_f(validation["rate"], validation["kma_transport_median"])

21.16

### lag features

#### for eval_df

In [42]:
shifted_week_rate = (
    train_df.group_by("date", "kma_pair", "transport_type")
    .agg(
        c("rate").median(),
    )
    .sort("date")
    .with_columns(
        c("rate")
        .rolling_median(window_size=7, center=False, min_samples=1)
        .over("kma_pair", "transport_type")
        .alias("rolling_rate"),
    )
    .with_columns(
        c("rolling_rate")
        .shift(1)
        .over("kma_pair", "transport_type")
        .alias("shifted_week_rate"),
    )
    .with_columns(
        c("shifted_week_rate").forward_fill().over("kma_pair", "transport_type")
    )
    .select(
        "date",
        "kma_pair",
        "transport_type",
        "shifted_week_rate",
    )
)

transformed_train_df = train_df.join(
    shifted_week_rate,
    on=["date", "kma_pair", "transport_type"],
    how="left",
)

In [23]:
transformed_train_df.shape

(289693, 19)

In [43]:
transformed_train_df.null_count()

rate,valid_miles,transport_type,weight,pickup_date,origin_kma,destination_kma,month,weekday,week,year,date,is_holiday_next_week,is_holiday,weight_mile,weight_per_mile,kma_pair,kma_transport_median,shifted_week_rate
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10005


In [27]:
transformed_train_df = transformed_train_df.drop_nulls()

In [33]:
def add_kma_pair_transport_lag_feature(
    train_df: pl.DataFrame,
    eval_df: pl.DataFrame,
) -> pl.DataFrame:

    cols = train_df.columns

    shifted_week_rate = (
        train_df.vstack(
            eval_df.with_columns(
                pl.lit(None).alias("rate"),
            ).select(cols)
        )
        .group_by("date", "kma_pair", "transport_type")
        .agg(
            c("rate").median(),
        )
        .sort("date")
        .with_columns(
            c("rate")
            .rolling_median(window_size=7, center=False, min_samples=1)
            .over("kma_pair", "transport_type")
            .alias("rolling_rate"),
        )
        .with_columns(
            c("rolling_rate")
            .shift(1)
            .over("kma_pair", "transport_type")
            .alias("shifted_week_rate"),
        )
        .with_columns(
            c("shifted_week_rate").forward_fill().over("kma_pair", "transport_type")
        )
        .select(
            "date",
            "kma_pair",
            "transport_type",
            "shifted_week_rate",
        )
    )

    transformed_eval_df = eval_df.join(
        shifted_week_rate,
        on=["date", "kma_pair", "transport_type"],
        how="left",
    )

    return transformed_eval_df

In [34]:
transformed_eval_df = add_kma_pair_transport_lag_feature(train_df, eval_df)

In [35]:
transformed_validation = add_kma_pair_transport_lag_feature(train, validation)

In [36]:
transformed_test = add_kma_pair_transport_lag_feature(train.vstack(validation), test)

In [37]:
metric = mape_f(
    transformed_eval_df.drop_nulls()["rate"],
    transformed_eval_df.drop_nulls()["shifted_week_rate"],
)
print("transformed_eval_df \t", metric)
metric = mape_f(
    transformed_validation.drop_nulls()["rate"],
    transformed_validation.drop_nulls()["shifted_week_rate"],
)
print("transformed_validation \t", metric)

transformed_eval_df 	 14.14
transformed_validation 	 12.02


In [38]:
transformed_eval_df.null_count()

rate,valid_miles,transport_type,weight,pickup_date,origin_kma,destination_kma,month,weekday,week,year,date,is_holiday_next_week,is_holiday,weight_mile,weight_per_mile,kma_pair,kma_transport_median,shifted_week_rate
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,173


In [39]:
transformed_validation.null_count()

rate,valid_miles,transport_type,weight,pickup_date,origin_kma,destination_kma,month,weekday,week,year,date,is_holiday_next_week,is_holiday,weight_mile,weight_per_mile,kma_pair,kma_transport_median,shifted_week_rate
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,180


### save dataframes

In [40]:
path = os.path.join("../", DATA_DIR, "train_df.parquet")
transformed_train_df.write_parquet(path)

path = os.path.join("../", DATA_DIR, "eval_df.parquet")
transformed_eval_df.write_parquet(path)

path = os.path.join("../", DATA_DIR, "validation.parquet")
transformed_validation.write_parquet(path)

path = os.path.join("../", DATA_DIR, "test.parquet")
transformed_test.write_parquet(path)